# Competition 2 - Final Report #
**Alex Cattie, Mike Szemenyei, Kerry Clarke**

### Part 1 ###

**Business Understanding - Framing the Analytical Question(s)**
- How accurately are we able to predict that a customer is going to default?
- What is the proabability that a customer is going to default? 
   - Not neccassirly 'yes' or 'no' but a percantage chance which would make classifying easier
- Is there a unique characteristic that those who defaul usually possess? 
- Can the probability of default produced through data mining represent the "real" probability of defaulting?
- Do those who default tend to be of a certain age? education level? or relationship?

**Data Understanding**



**Data Dictionary**

**EDA Graphs**

**Initial Modeling with Raw Data**
- Decision Tree
- Logistic Regression
- TPOT

### Part 2 ###

**Data Preprocessing**
- Pipeline 1
    - IQR
    - Min/Max
    - Skew
- Pipeline 2
    - Skew
    - 3stdev
    - z-score
- **Balancing the Target (initial notebook)**
- **Cross Validation vs. Train_Test_Split**

### Part 3 ###

**Models**
- Decistion Tree
- Logistic Regression
- SVM
- KNeighborsClassifier
- AdaBoost
- XGBoost
- Random Forest
- TPOT

**Most with and without Cross Validation**

### Part 4 ###

**Detailed Results/Explanation of the Model with the Best Results that we built**
- Explain any "tweaking" of the model
    - feature selection
    - playing with the parameters
- Why it may have been the best model
- This should be the best model/results we found by following the process from start to finish

### Part 5 ###

**TPOT Best Model/Results**
- Explain why we also ran a TPOT 
    - because it optimizes performance rather than reproducibility
- Explain the importance of knowing how and when to optimize one or the other

# Conclusion #